# Search Property Listings
1. Model Setting
2. Create API filter from user's query
3. Get Property Listings from Zillow API
4. Generate advice with the property listings

## 1. Model Setting - Gemini pro 1.0

In [4]:
# %pip list | grep google-cloud-aiplatform
# %pip list | grep google-api-core

In [5]:
# %pip install google-cloud-aiplatform==1.43.0
# %pip install google-api-core==2.17.1

In [30]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession, Part
import vertexai.preview.generative_models as generative_models

In [31]:
vertexai.init(project="adsp-capstone-property-pilot", location="us-central1")

In [32]:
model = GenerativeModel("gemini-1.0-pro")
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

### Test

In [33]:
prompt = "Hello."
print(get_chat_response(chat, prompt))

Hello! I'm glad you're here. What can I do for you today?


## 2. Create API filter from user's query

In [34]:
def generate_prompt_apifilter(instruction, user_query):
    return instruction.replace("{USER_QUERY}", user_query)

In [35]:
instruction = """
### Instructions ###
I want you to act as a principal software engineer. You'll be given a task to determine the parameters of an API call based on the "User's query" below, 
please modify the conditions and create the appropriate API filter. Provide your answer only for the part within ‘{}’. 
Do not include any other explanations in your response. Read the below instructions.

— Filter Options (Optional, so not required if not specified in the User's query) —

1. location
Format: STRING(lowercase)
Example: santa monica, ca

*If not specifically mentioned, use 'chicago, il'.
*Do not alter the format from the above example. 
*If you want to search for a specific neighborhood like Chicago's West Loop, please specify it in the Filter option '16. keywords', not here.
Incorrect example: "location": "west loop, chicago, il"
Correct example: "location": "chicago, il", "keywords": "west loop"


2. status_type
For purchase or for rent
Format: STRING
Example:
ForSale
ForRent

3. home_type
Property type comma-separated or empty for all types
Format: STRING
Example(ForRent):
Townhomes
Houses
Apartments_Condos_Co-ops

Example(ForSale):
Multi-family
Apartments
Houses
Manufactured
Condos
LotsLand
Townhomes

*If it cannot be determined from the User's query, specification is not necessary.

4. minPrice
If status_type = ForSale you can filter by min price.
Format: NUMBER
Example: 200000

5. maxPrice
If status_type = ForSale you can filter by max price.
Format: NUMBER
Example: 900000

6. rentMinPrice
If status_type = ForRent you can filter by min rent price.
Format: NUMBER
Example: 2000

7. rentMaxPrice
If status_type = ForRent you can filter by max rent price.
Format: NUMBER
Example: 3000

8. bathsMin
Bathrooms min count
Format: NUMBER
Example: 2
* If a specific number of baths is specified, rather than a range, please enter the same number for both bathsMin and bathsMax.

9. bathsMax
Bathrooms max count
Format: NUMBER
Example: 3

10. bedsMin
Bedrooms min count
Format: NUMBER
Example: 1
* If a specific number of bedrooms is specified, rather than a range, please enter the same number for both bedsMin and bedsMax.

11. bedsMax
Bedrooms max count
Format: NUMBER
Example: 3

12. sqftMin
Square Feet min value
Format: NUMBER
Example: 600

13. sqftMax
Square Feet max value
Format: NUMBER
Example: 1500

14. buildYearMin
Year Built min value
Format: NUMBER
Example: 1980

15. buildYearMax
Year Built max value
Format: NUMBER
Example: 2023

16. keywords
Filter with keywords.
Format: STRING
Example1: ‘gym’
Example2: ‘West Loop’
Example3: ‘pet, pool, South Loop‘
*Not necessary if already specified in another filter or if there is no clear specification in user’s query. Specify conditions that cannot be specified in other filters by using keywords.

17. sort
Sorting criteria for the results
Always specify "Newest"


Now, use the following query to determine the appropriate parameters.

### Example query ###
I am looking for a one-bedroom.

### Example Output ###
{
"location":"chicago, il",
"bedsMin":"1",
"bedsMax":"1",
"sort":"Newest"
}

### Example query ###
I am looking for a rental in South Loop, Chicago, with at least one bedroom and a rent of $3000 or less per month.

### Example Output ###
{
"location": "chicago, il",
"status_type": "ForRent",
"rentMaxPrice": 3000,
"bedsMin": 1,
"keywords": "South Loop",
"sort":"Newest"
}

### Example query ###
I am looking for a rental in West Loop, Chicago, with one bedroom and a rent between $2000 and $2500.

### Example Output ###
{
"location": "chicago, il",
"status_type": "ForRent",
"rentMinPrice": 2500,
"rentMaxPrice": 3000,
"bedsMin": 1,
"bedsMax": 1,
"keywords": "West Loop",
"sort":"Newest"
}

### Example query ###
I want to buy a house in River North with at least two bedrooms.

### Example Output ###
{
"location":"chicago, il",
"status_type": "ForSale",
"keywords":"River North",
"bedsMin":"2",
"sort":"Newest"
}

### Example query ###
I am looking for a rental around Wicker Park in Chicago with a gym and a pool. It should be built within the last 10 years.

### Example Output ###
{
"location": "chicago, il",
"home_type": "Apartments_Condos_Co-ops",
"status_type": "ForRent",
"buildYearMin": 2014,
"keywords": "Wicker Park, gym, pool",
"sort":"Newest"
}

### Example query ###
I am looking for a rental with a balcony, over 700 sqft in size, and less than 5 years old, located near downtown.

### Example Output ###
{
"location": "chicago, il",
"status_type": "ForRent",
"sqftMin":"700",
"buildYearMin": 2019,
"keywords": "downtown, balcony",
"sort":"Newest"
}


### User’s query ###
{USER_QUERY}

### Output ###
"""

In [36]:
user_query = "I am looking for a two-bedroom."
print(generate_prompt_apifilter(instruction, user_query))


### Instructions ###
I want you to act as a principal software engineer. You'll be given a task to determine the parameters of an API call based on the "User's query" below, 
please modify the conditions and create the appropriate API filter. Provide your answer only for the part within ‘{}’. 
Do not include any other explanations in your response. Read the below instructions.

— Filter Options (Optional, so not required if not specified in the User's query) —

1. location
Format: STRING(lowercase)
Example: santa monica, ca

*If not specifically mentioned, use 'chicago, il'.
*Do not alter the format from the above example. 
*If you want to search for a specific neighborhood like Chicago's West Loop, please specify it in the Filter option '16. keywords', not here.
Incorrect example: "location": "west loop, chicago, il"
Correct example: "location": "chicago, il", "keywords": "west loop"


2. status_type
For purchase or for rent
Format: STRING
Example:
ForSale
ForRent

3. home_type
Prope

### Test

In [37]:
user_query = "I am looking for a two-bedroom."
prompt = generate_prompt_apifilter(instruction, user_query)
print(get_chat_response(chat, prompt))

{
"bedsMin": "2",
"bedsMax": "2",
"status_type": "ForRent",
"location":"chicago, il",
"sort":"Newest"
}


In [38]:
user_query = "I am looking for a rental in Fulton Market, Chicago, with at least one bedroom and a rent of $3500 or less per month."
prompt = generate_prompt_apifilter(instruction, user_query)
print(get_chat_response(chat, prompt))

{
"location": "chicago, il",
"status_type": "ForRent",
"rentMaxPrice": 3500,
"bedsMin": 1,
"keywords": "Fulton Market",
"sort":"Newest"
}


In [39]:
user_query = "I want to buy a house in Chinatown, Chicago with a minimum of 2 bathrooms and 2 bedrooms."
prompt = generate_prompt_apifilter(instruction, user_query)
print(get_chat_response(chat, prompt))

{
"location":"chicago, il",
"status_type": "ForSale",
"keywords":"Chinatown",
"bathsMin":"2",
"bedsMin":"2",
"sort":"Newest"
}


In [40]:
user_query = "I am looking for a rental around Hyde Park with a gym and a pool. It should be built within the last 15 years."
prompt = generate_prompt_apifilter(instruction, user_query)
print(get_chat_response(chat, prompt))

{
"location":"chicago, il",
"status_type": "ForRent",
"home_type": "Apartments_Condos_Co-ops",
"buildYearMin": 2009,
"keywords": "Hyde Park, gym, pool",
"sort":"Newest"
}


In [41]:
user_query = "I am looking for a rental in the Old Town area of Chicago with a balcony and a pool."
prompt = generate_prompt_apifilter(instruction, user_query)
print(get_chat_response(chat, prompt))

{
"location": "chicago, il",
"home_type": "Apartments_Condos_Co-ops",
"status_type": "ForRent",
"keywords": "Old Town, balcony, pool",
"sort":"Newest"
}


## 3. Get Property Listings from Zillow API

### Example - Extended Search endpoint

In [18]:
import requests

url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

headers = {
	"X-RapidAPI-Key": "YOUR_API_KEY",
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

In [19]:
querystring = {"location":"chicago, il", "status_type":"ForRent", "bathsMin":"1","bedsMin":"3", "keywords":"west", "sort":"Newest"}
response = requests.get(url, headers = headers, params = querystring)

In [20]:
response.json()

{'props': [{'dateSold': None,
   'propertyType': 'APARTMENT',
   'lotAreaValue': None,
   'address': '4734 S Kildare Ave #2, Chicago, IL 60632',
   'variableData': None,
   'unit': '# 2',
   'zestimate': None,
   'imgSrc': 'https://photos.zillowstatic.com/fp/3e7358cce3b8100167e5293b511f0dc5-p_e.jpg',
   'price': 1575,
   'detailUrl': '/homedetails/4734-S-Kildare-Ave-2-Chicago-IL-60632/2058742577_zpid/',
   'bedrooms': 3,
   'contingentListingType': None,
   'longitude': -87.73125,
   'latitude': 41.806747,
   'listingStatus': 'FOR_RENT',
   'zpid': '2058742577',
   'listingSubType': {},
   'rentZestimate': 1634,
   'daysOnZillow': 0,
   'bathrooms': 1,
   'livingArea': 1100,
   'country': 'USA',
   'currency': 'USD',
   'lotAreaUnit': None,
   'hasImage': True},
  {'dateSold': None,
   'propertyType': 'APARTMENT',
   'lotAreaValue': None,
   'address': '6651 N Rockwell St #2, Chicago, IL 60645',
   'variableData': None,
   'unit': '# 2',
   'zestimate': None,
   'imgSrc': 'https://phot

In [21]:
def extract_properties(response, n, fields):
    """
    Extracts and returns the top 'n' properties from a given API response
    based on the specified fields, appending the base URL to 'detailUrl'.
    
    Parameters:
    - response: The response object from an API call.
    - n: The number of top properties to extract.
    - fields: A list of strings representing the fields to extract from each property.
    
    Returns:
    - A list of dictionaries, each representing a property with only the specified fields.
    """
    base_url = "https://www.zillow.com"
    try:
        # Convert the response to JSON format if it's not already a dictionary
        if not isinstance(response, dict):
            response = response.json()

        # Extract the top 'n' properties based on the specified fields
        top_properties = []
        for prop in response['props'][:n]:
            extracted_prop = {field: (base_url + prop['detailUrl'] if field == 'detailUrl' else prop.get(field, None)) for field in fields}
            top_properties.append(extracted_prop)

        return top_properties

    except Exception as e:
        print("An error occurred:", e)
        return []  # Return an empty list in case of an error


In [22]:
# Define fields to extract
fields = ["propertyType", "address", "price", "bedrooms", "bathrooms", "detailUrl", "imgSrc"]

In [23]:
# Get top 3 properties with selected fields
top_properties = extract_properties(response, 3, fields)
top_properties

[{'propertyType': 'APARTMENT',
  'address': '4734 S Kildare Ave #2, Chicago, IL 60632',
  'price': 1575,
  'bedrooms': 3,
  'bathrooms': 1,
  'detailUrl': 'https://www.zillow.com/homedetails/4734-S-Kildare-Ave-2-Chicago-IL-60632/2058742577_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/3e7358cce3b8100167e5293b511f0dc5-p_e.jpg'},
 {'propertyType': 'APARTMENT',
  'address': '6651 N Rockwell St #2, Chicago, IL 60645',
  'price': 2100,
  'bedrooms': 3,
  'bathrooms': 1,
  'detailUrl': 'https://www.zillow.com/homedetails/6651-N-Rockwell-St-2-Chicago-IL-60645/2071905782_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/15b92afe0366d891393d59a5866307b3-p_e.jpg'},
 {'propertyType': 'APARTMENT',
  'address': '3826 W Monroe St, Chicago, IL 60624',
  'price': 1650,
  'bedrooms': 3,
  'bathrooms': 1,
  'detailUrl': 'https://www.zillow.com/homedetails/3826-W-Monroe-St-Chicago-IL-60624/2076380588_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/1628de89aa481ce8956c7328a1e016af-

### Example - Property details endpoint

In [48]:
top_properties[0]['detailUrl']

'https://www.zillow.com/homedetails/10500-S-Church-St-Chicago-IL-60643/4143638_zpid/'

In [56]:
# Endpoint
url_details = "https://zillow-com1.p.rapidapi.com/property"

# URL example
property_url = top_properties[0]['detailUrl']

querystring = {"property_url":property_url}
response = requests.get(url_details, headers = headers, params = querystring)

In [62]:
response = response.json()
response['description']

'Location, Location, Location! Make this huge corner bungalow your dream home in the Beverly area.  Step inside and be engulfed by the entryway, that leads to a unique living room with a custom full wall book shelf, that flows into an elegant dining area with a beautiful chandelier and complemented by a huge bay window. Then move to the completely remodeled kitchen that will blow you away.  The bath has been updated with ceramic tile and vanity.  The lower level  features a possible 3rd bedroom, an office, storage or exercise room, and a roughed-in bath.  The attic is ready for your creative ideas for a master suite or two additional bedrooms and a bath.  Walk out  to the huge deck that is great for entertaining.   Proceed to a newer two car garage.from the large yard enclosed by a wrought iron fence. Near Metra, bus line and expressway.  Beautiful, quiet block with nice neighbors. This beauty will not last.'

In [64]:
def fetch_descriptions(properties):
    """
    Fetches the descriptions of each property and appends them to the list.

    Parameters:
    - properties: A list of dictionaries representing properties.

    Returns:
    - A list of dictionaries(the same format as input).
    """
    url_details = "https://zillow-com1.p.rapidapi.com/property"

    for prop in properties:
        # Fetch detailed information using detailUrl
        querystring = {"property_url": prop['detailUrl']}
        response = requests.get(url_details, headers=headers, params=querystring)

        # Convert to JSON and fetch description
        detail_data = response.json()
        prop['description'] = detail_data.get('description', 'No description available')

    return properties

In [65]:
fetch_descriptions(top_properties)

[{'propertyType': 'SINGLE_FAMILY',
  'address': '10500 S Church St, Chicago, IL 60643',
  'price': 244900,
  'bedrooms': 2,
  'bathrooms': 1,
  'detailUrl': 'https://www.zillow.com/homedetails/10500-S-Church-St-Chicago-IL-60643/4143638_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/a5a3f713652b246545f67e89d17d0ed0-p_e.jpg',
  'description': 'Location, Location, Location! Make this huge corner bungalow your dream home in the Beverly area.  Step inside and be engulfed by the entryway, that leads to a unique living room with a custom full wall book shelf, that flows into an elegant dining area with a beautiful chandelier and complemented by a huge bay window. Then move to the completely remodeled kitchen that will blow you away.  The bath has been updated with ceramic tile and vanity.  The lower level  features a possible 3rd bedroom, an office, storage or exercise room, and a roughed-in bath.  The attic is ready for your creative ideas for a master suite or two additional bedroo

### For main use case

In [66]:
import json

In [67]:
user_query = "I am looking for a two-bedroom."
prompt = generate_prompt_apifilter(instruction, user_query)
print(get_chat_response(chat, prompt))

{
"location":"chicago, il",
"bedsMin": "2",
"bedsMax": "2",
"sort":"Newest"
}


In [68]:
# get an api filter through Gemini
querystring = get_chat_response(chat, prompt)

# from str to dict
querystring = json.loads(querystring)

# get listings from zillow api
response = requests.get(url, headers = headers, params = querystring)

In [69]:
# Define fields to extract
fields = ["propertyType", "address", "price", "bedrooms", "bathrooms", "detailUrl", "imgSrc"]

In [70]:
# Get top 3 properties with selected fields
top_properties = extract_properties(response, 3, fields)
top_properties

[{'propertyType': 'SINGLE_FAMILY',
  'address': '10500 S Church St, Chicago, IL 60643',
  'price': 244900,
  'bedrooms': 2,
  'bathrooms': 1,
  'detailUrl': 'https://www.zillow.com/homedetails/10500-S-Church-St-Chicago-IL-60643/4143638_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/a5a3f713652b246545f67e89d17d0ed0-p_e.jpg'},
 {'propertyType': 'CONDO',
  'address': '1660 N La Salle St #1302, Chicago, IL 60614',
  'price': 465000,
  'bedrooms': 2,
  'bathrooms': 2,
  'detailUrl': 'https://www.zillow.com/homedetails/1660-N-La-Salle-St-1302-Chicago-IL-60614/348746300_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/5b24f7400972354118c83732038a7aeb-p_e.jpg'},
 {'propertyType': 'MULTI_FAMILY',
  'address': '7801 W Summerdale Ave, Chicago, IL 60656',
  'price': 325500,
  'bedrooms': 2,
  'bathrooms': 2,
  'detailUrl': 'https://www.zillow.com/homedetails/7801-W-Summerdale-Ave-Chicago-IL-60656/3583976_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/3d472df8b68f814af81000

In [73]:
top_properties = fetch_descriptions(top_properties)
top_properties

[{'propertyType': 'SINGLE_FAMILY',
  'address': '10500 S Church St, Chicago, IL 60643',
  'price': 244900,
  'bedrooms': 2,
  'bathrooms': 1,
  'detailUrl': 'https://www.zillow.com/homedetails/10500-S-Church-St-Chicago-IL-60643/4143638_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/a5a3f713652b246545f67e89d17d0ed0-p_e.jpg',
  'description': 'Location, Location, Location! Make this huge corner bungalow your dream home in the Beverly area.  Step inside and be engulfed by the entryway, that leads to a unique living room with a custom full wall book shelf, that flows into an elegant dining area with a beautiful chandelier and complemented by a huge bay window. Then move to the completely remodeled kitchen that will blow you away.  The bath has been updated with ceramic tile and vanity.  The lower level  features a possible 3rd bedroom, an office, storage or exercise room, and a roughed-in bath.  The attic is ready for your creative ideas for a master suite or two additional bedroo

## 4. Generate advice with the property listings

In [74]:
def generate_prompt_property(instruction, user_query, property_info):
    updated_instruction = instruction.replace("{USER_QUERY}", user_query)
    updated_instruction = updated_instruction.replace("{PROPERTY_INFO}", property_info)
    return updated_instruction

In [75]:
instruction = """
### Instructions ###
I want you to act as a real estate advisor. Please recommend properties to the customer. 
'User’s query' represents the questions you are asked by the customer. 
'Real estate properties to present to the user' are the properties you will introduce to the customer. Please include all the details provided, including links, in your response.

### User’s query ###
{USER_QUERY}

### Real estate properties to present to the user ###
{PROPERTY_INFO}

### Output ###
"""

In [76]:
def format_properties(properties, fields):
    formatted_properties = []
    for prop in properties:
        # Format and concatenate each field of each property
        formatted_prop = ', '.join(f"{field.capitalize().replace('_', ' ')}: {prop.get(field, 'N/A')}" for field in fields)
        formatted_properties.append(formatted_prop)
    return '\n\n'.join(formatted_properties)

In [80]:
# Change format
fields_desc = fields + ['description']
property_info = format_properties(top_properties, fields_desc)
print(property_info)

Propertytype: SINGLE_FAMILY, Address: 10500 S Church St, Chicago, IL 60643, Price: 244900, Bedrooms: 2, Bathrooms: 1, Detailurl: https://www.zillow.com/homedetails/10500-S-Church-St-Chicago-IL-60643/4143638_zpid/, Imgsrc: https://photos.zillowstatic.com/fp/a5a3f713652b246545f67e89d17d0ed0-p_e.jpg, Description: Location, Location, Location! Make this huge corner bungalow your dream home in the Beverly area.  Step inside and be engulfed by the entryway, that leads to a unique living room with a custom full wall book shelf, that flows into an elegant dining area with a beautiful chandelier and complemented by a huge bay window. Then move to the completely remodeled kitchen that will blow you away.  The bath has been updated with ceramic tile and vanity.  The lower level  features a possible 3rd bedroom, an office, storage or exercise room, and a roughed-in bath.  The attic is ready for your creative ideas for a master suite or two additional bedrooms and a bath.  Walk out  to the huge dec

In [81]:
# generate prompt
prompt = generate_prompt_property(instruction, user_query, property_info)

In [82]:
print(prompt)


### Instructions ###
I want you to act as a real estate advisor. Please recommend properties to the customer. 
'User’s query' represents the questions you are asked by the customer. 
'Real estate properties to present to the user' are the properties you will introduce to the customer.　Please include all the details provided, including links, in your response.

### User’s query ###
I am looking for a two-bedroom.

### Real estate properties to present to the user ###
Propertytype: SINGLE_FAMILY, Address: 10500 S Church St, Chicago, IL 60643, Price: 244900, Bedrooms: 2, Bathrooms: 1, Detailurl: https://www.zillow.com/homedetails/10500-S-Church-St-Chicago-IL-60643/4143638_zpid/, Imgsrc: https://photos.zillowstatic.com/fp/a5a3f713652b246545f67e89d17d0ed0-p_e.jpg, Description: Location, Location, Location! Make this huge corner bungalow your dream home in the Beverly area.  Step inside and be engulfed by the entryway, that leads to a unique living room with a custom full wall book shelf, t

In [83]:
# get final response from Gemini
print(get_chat_response(chat, prompt))

## Two-Bedroom Recommendations for You:

Based on your interest in two-bedroom properties, here are three options that might be a good fit for you:

**1. Spacious Bungalow in Beverly (Single Family):**

* **Address:** 10500 S Church St, Chicago, IL 60643
* **Price:** $244,900
* **Bedrooms:** 2
* **Bathrooms:** 1
* **Details:** https://www.zillow.com/homedetails/10500-S-Church-St-Chicago-IL-60643/4143638_zpid/
* **Image:** https://photos.zillowstatic.com/fp/a5a3f713652b246545f67e89d17d0ed0-p_e.jpg
* **Description:** This charming corner bungalow boasts a spacious living room with a custom bookshelf, an elegant dining area with a bay window, and a completely remodeled kitchen. It also features a finished basement with a possible third bedroom, an office, and storage space. The attic is perfect for creating a master suite or two additional bedrooms and a bathroom. Enjoy the outdoors on the large deck and fenced-in yard. This home is near Metra, the bus line, and the expressway, making it 

## 5. Integrating all functions

In [2]:
## TBD